# Exercice 1 : Manipulation de points sur une courbe elliptique


In [6]:
# Question 1
Zp = Zmod(29)
E_1 = EllipticCurve(Zp, [-3, 5])


print(f"curve order: {E_1.order()}")
for i in range(5):
    P = E_1.random_point()
    print(P)
    print(f"point order: {P.order()}")

curve order: 24
(26 : 4 : 1)
point order: 4
(3 : 20 : 1)
point order: 12
(5 : 12 : 1)
point order: 12
(20 : 12 : 1)
point order: 6
(15 : 0 : 1)
point order: 2


On remarque que l’ordre du point divise celui du groupe. Ici, le groupe est d’ordre 24, donc l’ordre du point généré peut être 1, 2, 4, 6, 12 ou 24 (ici il n’y a pas de point d’ordre 24).

### Question 2
Il suffit de suivre les instructions, en se rappelant que les opérations sont faites sur le corps finis

In [7]:
def double_point(P, E):
    if P.is_zero():
        return P
    if P[1] == 0:
        retu   rn E((0,1,0))
    Zp = E.base_field()
    lamb = Zp((3*P[0]*P[0] + E.a4())/(2*P[1]))
    x = lamb**2 - 2*P[0]
    y = lamb*(P[0]-x) - P[1]
    
    return E((x,y))

# add_points handles all cases here. Namely, it calls double_points if P1 == P2
def add_points(P1, P2, E):
    if P1.is_zero():
        return P2
    if P2.is_zero():
        return P1
    if P1 == P2:
        return double_point(P, E)
    if P1 == -P2:
        return E((0,1,0))
    Zp = E.base_field()
    lamb = Zp((P2[1] - P1[1])/(P2[0] - P1[0]))
    x = lamb**2 - P1[0] - P2[0]
    y = lamb*(P1[0]-x) - P1[1]
    
    return E((x,y))

def scalar_mult(k, P, E):
    res = E((0,1,0))
    for bit in ZZ(k).bits()[::-1]:
        res = double_point(res, E)
        if bit == 1:
            # carefull, here add_point handle the case res == E, otherwise
            # we need to check this edge case
            res = add_points(res, P, E)
    return res

Zp = Zmod(29)
E_1 = EllipticCurve(Zp, [-3, 5])
for P in E_1:
    for Q in E_1:
        if P == Q:
            res =  double_point(P, E_1)
        else:
            res = add_points(P, Q, E_1)
        if res != P+Q:
            print(f"Error for {P}+{Q}")
    for k in range(P.order()):
        if k*P != scalar_mult(k, P, E_1):
            print(f"Error for {k}*{P}")

### Question 3
La façon la plus naive de faire (qui ne passe pas à l'échelle sur les courbes utilisées en cryptographie en pratique) est de simplement additionner le points jusqu'à tomber sur le point infini.

Une façon un peu plus "légère" est de ne tester que les ordres __possibles__, c'est à dire les diviseurs de l'ordre de la courbe.

In [1]:
def get_point_order(P, E):
    for k in E.order().divisors():
        if k*P == E.point((0,1,0)):
            return k

Zp = Zmod(29)
E_1 = EllipticCurve(Zp, [-3, 5])
for P in E_1:
    if get_point_order(P, E_1) != P.order():
        print("Error")

### Question 4
Ici on remarque que pour la courbe $E_2$, qui est d'ordre 31 (un nombre premier), l'ordre des chacun des points est 31 (sauf le point infini). C'est pour ça qu'on choisi généralement des courbes d'ordre premier, ou qu'on prend un sous-groupe de points d'ordre premier (c'est à dire qu'on prend un point de base avec un ordre premier).

In [9]:
def is_point_x(x, E):
    # We make sure to consider x in Zp
    Zp = E.base_field()
    x = Zp(x)
    tmp = x**3 + E.a4()*x + E.a6()
    return tmp.is_square(), tmp

# Equivalent to E.points()
def gen_all_points(E):
    # Don't forget infinity point
    points = [E.point((0, 1, 0))]
    Zp = E.base_ring()
    p = Zp.order()
    for x in range(p):
        valid, y2 = is_point_x(x, E)
        if valid:
            y = y2.sqrt()
            points.append(E.point((x, y)))
            if y != 0:
                points.append(E.point((x, -y)))
    return points

Zp = Zmod(29)

E_1 = EllipticCurve(Zp, [-3, 5])
print("Points in E_1")
for P in gen_all_points(E_1):
    print(f"{P} order {P.order()}")
    
E_2 = EllipticCurve(Zp, [-2, 20])
print("\nPoints in E_2")
for P in gen_all_points(E_2):
    print(f"{P} order {P.order()}")

Points in E_1
(0 : 1 : 0) order 1
(0 : 11 : 1) order 12
(0 : 18 : 1) order 12
(2 : 6 : 1) order 3
(2 : 23 : 1) order 3
(3 : 9 : 1) order 12
(3 : 20 : 1) order 12
(4 : 12 : 1) order 4
(4 : 17 : 1) order 4
(5 : 12 : 1) order 12
(5 : 17 : 1) order 12
(6 : 0 : 1) order 2
(8 : 0 : 1) order 2
(15 : 0 : 1) order 2
(16 : 14 : 1) order 6
(16 : 15 : 1) order 6
(17 : 13 : 1) order 12
(17 : 16 : 1) order 12
(20 : 12 : 1) order 6
(20 : 17 : 1) order 6
(26 : 4 : 1) order 4
(26 : 25 : 1) order 4
(28 : 6 : 1) order 6
(28 : 23 : 1) order 6

Points in E_2
(0 : 1 : 0) order 1
(0 : 7 : 1) order 31
(0 : 22 : 1) order 31
(2 : 13 : 1) order 31
(2 : 16 : 1) order 31
(7 : 1 : 1) order 31
(7 : 28 : 1) order 31
(8 : 9 : 1) order 31
(8 : 20 : 1) order 31
(9 : 8 : 1) order 31
(9 : 21 : 1) order 31
(11 : 13 : 1) order 31
(11 : 16 : 1) order 31
(12 : 10 : 1) order 31
(12 : 19 : 1) order 31
(13 : 4 : 1) order 31
(13 : 25 : 1) order 31
(15 : 1 : 1) order 31
(15 : 28 : 1) order 31
(16 : 13 : 1) order 31
(16 : 16 : 1) o

### Question 5

Ici l'ordre $\ell$ de la courbe se factorise en $743 \times 69511034407065278854542476309161687$, il n'est donc pas premier. L'ordre d'un point peut prendre n'importe quel diviseur de $\ell$.

Ici le point $P$ est d'ordre 743, ce qui est clairement insuffisant pour apporter une quelconque sécurité au regard du problème du logarithme discret : un attaquant peut simplement tester les 742 valeurs de $k$ pour retrouver la valeur secrète.

Utiliser un point d'ordre $\ell$ n'est pas suffisant non plus. Les algorithme génériques pour résoudre ECDLP tournent en $\mathcal{O}(\sqrt n)$. Ici $n = \ell \approx 2^{125}$. Une telle coubre offre donc, au mieux, $125/2 = 62.5$ bits de sécurité (on cherche au minimum 128 bits en cryptographie moderne).


In [10]:
Zp = Zmod(51646698564449502183630508998684683453)
E_3 = EllipticCurve(Zp, [-3, 6])
print("l =", E_3.order(), "=", factor(E_3.order()))
print("Ordre possible d'un point: ", E_3.order().divisors())

P = E_3((5866391592011188692732729142407841644, 50082992786731864883063206566493560050))
print(P.order())
print("order bit length:", len(E_3.order().bits()))

l = 51646698564449502188925059897707133441 = 743 * 69511034407065278854542476309161687
Ordre possible d'un point:  [1, 743, 69511034407065278854542476309161687, 51646698564449502188925059897707133441]
743
order bit length: 126


# Exercice 2 : ECDSA

Multiples sources disponibles en ligne. Gardez en tête que [intégrité $\neq$ authenticité](https://security.stackexchange.com/questions/93322/difference-between-authentication-integrity-and-data-origin-authentication).

L’idée est que ECDSA est une fonction de signature, permettant d’authentifier un message. C’est à dire qu’un utilisateur ayant une clé $(d, P)$, avec $d$ privé et $P = d \times G$ publique, peut signer
un message $m$ à l’aide de sa clé privée. La signature consiste en deux entiers $(r, s)$, considérés publiques.
Tout utilisateur en possession du message $m$, de la clé publique $P$ et de la signature $(r, s)$ est en mesure de vérifier si la signature est valide.




In [11]:
from hashlib import sha256

def ecdsa_sign(m, d, n, G):
    h = int(sha256(m.encode()).digest().hex(), 16)
    r = 0
    s = 0
    while s == 0:
        while r == 0:
            k = Integer(randint(1, n))    # Should be a call to a CSPRNG, not randint
            Q = k*G
            r = int(Q[0]) % n
        s = int(k.inverse_mod(n) * (h + d*r)) % n
    return r, s

def ecdsa_verify(m, r, s, P, n, G, curve):
    # Check the public key
    if P.is_zero() or not P in curve or not (n*P).is_zero():
        return False

    if not (1 <= r,s < n):
        return False

    h = int(sha256(m.encode()).digest().hex(), 16)
    s_inv = s.inverse_mod(n)
    scal1 = (h * s_inv) % n
    scal2 = (r * s_inv) % n
    Q = scal1*G + scal2*P

    return Q[0] == r % n

# NIST's P-256 parameters
p = 2**256 - 2**224 + 2**192 + 2**96 - 1 #  prime defining the field
# y^ = x^3 + ax + b
a = -3
b = 41058363725152142129326129780047268409114441015993725554835256314039467401291
Gx = 0x6B17D1F2E12C4247F8BCE6E563A440F277037D812DEB33A0F4A13945D898C296
Gy = 0x4FE342E2FE1A7F9B8EE7EB4A7C0F9E162BCE33576B315ECECBB6406837BF51F5
n = 115792089210356248762697446949407573529996955224135760342422259061068512044369 # G's order

# Create the curve and the generator on sage
P256 = EllipticCurve(Zmod(p), [a, b])
G = P256([Gx, Gy])

# Private key, use appropriate CSPRNG in practice! 
d = randint(2, n-1)
P = d*G

m = "Test message"
r, s = ecdsa_sign(m, d, n, G)
if not ecdsa_verify(m, r, s, P, n, G, P256):
    print("Error in verification")

Le nonce est effictivement sensible. Il doit être secret, et différent pour chaque signature effectuée avec la même clé. Toute fuite d’information (même quelques bits) sur le nonce compromet la confidentialité de la clé privée.

# Exercice 3 : Attaques sur ECDSA 

Définissons les notations suivantes :

— $k$ est le nonce

— $G$ est le point générateur, publique, utilisé par tout les utilisateur de la courbe $P256$. $G$ est d’ordre $n$ et $P256$ est définie sur $Z/pZ$.

— La bi-clé d’un utilisateur $A$ est notée $(d_A, P_A)$ avec $P_A = d_A × G$.

— Si $P = (x, y)$ est un point sur la courbe, $Px$ désigne sa coordonée en $x$.


Si le nonce est connu par l'attaquant, il peut directement retrouver la clé privée :

Sachant que $s = k^{-1}(h(m) + d\times r) \bmod n$, la seule inconnue est $d$, la clé privée. On peut simplement inverser les opérations une par une pour arriver au résultat (toute les opération si dessous sont effectuées modulo $n$):
			\begin{align*}
				& s = k^{-1}(h(m) + d\times r)\\
				& s\times k = h(m) + d\times r\\
				& s\times k - h(m) = d\times r\\
				& (s\times k - h(m))r^{-1} = d
			\end{align*}

In [12]:
from hashlib import sha256

# NIST's P-256 parameters
p = 2**256 - 2**224 + 2**192 + 2**96 - 1 #  prime defining the field
# y^ = x^3 + ax + b
a = -3
b = 41058363725152142129326129780047268409114441015993725554835256314039467401291
Gx = 0x6B17D1F2E12C4247F8BCE6E563A440F277037D812DEB33A0F4A13945D898C296
Gy = 0x4FE342E2FE1A7F9B8EE7EB4A7C0F9E162BCE33576B315ECECBB6406837BF51F5
n = 115792089210356248762697446949407573529996955224135760342422259061068512044369 # G's order

# Create the curve and the generator on sage
P256 = EllipticCurve(Zmod(p), [a, b])
G = P256([Gx, Gy])

# Public key to verify the signatures 
P = P256(0x7453873d3b072c061d03c3d09aba344b6a25f406a3dad40b0281025824bda4a3, 0x9e41b56ecaaf199fbb6d33fb2d748ec454c128aa7be74a1998cf57de34b7988b)

m1 = "Message important"
r1 = 0x62286e65a6703cdff07b19dba56c34c84d9be61edfac86b347d7fc337615a3bf
s1 = 0xc4926a0932e3d1f6aeec3c6a55926aaa73911c2b7a1325b2cc2e45ed70271578
k1 = 0x7963d7dadb13b6c53864c267dad2e94e52c00e41eaabd03c6f5c559ef912722b

# Knowing k, we can simply reverse the operations to recover d
h = int(sha256(m1.encode()).digest().hex(), 16)
d1 = (s1 * k1 - h )*r1.inverse_mod(n) % n
print("The private key is", hex(d1))

if d1*G == P:
    print("The private key we found fits the public key P")
else:
    print("The private key we found does not fit the public key P")

The private key is 0xe5f5a8f342537d97da4a1c6ff7014c1b4a9167c46e05779da1bcf172d49602a1
The private key we found fits the public key P


Si un nonce est réutilisé, on peut s'en rendre compte facilement en regardant la valeur de $r$ dans les signatures. En effet, comme $r$ est la coordonnée en $x$ de $k\times P$, utiliser le même nonce donne la même valeur de $r$.

À partir de là, l'attaquant peut retrouver la clé privée à partir des deux messages et des signatures correspondantes.

On sait que $r_1 = r_2 = r$. $s_1 = k^{-1}(h(m_1) + d\times r)$ et $s_2 = k^{-1}(h(m_2) + d\times r)$.
	En soustrayant les deux valeurs de $s_1$ et $s_2$, on peut retrouver la valeur du nonce. À partir de là, la même attaque s'applique.
    
Les opérations ci-dessous sont toutes effectuées modulo $n$.
			\begin{align*}
				s_1 -s_2 & = k^{-1}(h(m_1) + d\times r) -  k^{-1}(h(m_2) + d\times r) \\
				& = k^{-1}(h(m_1) -  h(m_2))\\
				\Rightarrow k & = (s_1 -s_2)^{-1} \times (h(m_1)-h(m_2))
			\end{align*}

In [13]:
from hashlib import sha256

def ecdsa_sign(m, d, n, G):
    h = int(sha256(m.encode()).digest().hex(), 16)
    r = 0
    s = 0
    while s == 0:
        while r == 0:
            k = Integer(randint(1, n))    # Should be a call to a CSPRNG, not randint
            Q = k*G
            r = int(Q[0]) % n
        s = int(k.inverse_mod(n) * (h + d*r)) % n
    return r, s

def ecdsa_verify(m, r, s, P, n, G, curve):
    # Check the public key
    if P.is_zero() or not P in curve or not (n*P).is_zero():
        return False

    if not (1 <= r,s < n):
        return False

    h = int(sha256(m.encode()).digest().hex(), 16)
    s_inv = s.inverse_mod(n)
    scal1 = (h * s_inv) % n
    scal2 = (r * s_inv) % n
    Q = scal1*G + scal2*P

    return Q[0] == r % n

# NIST's P-256 parameters
p = 2**256 - 2**224 + 2**192 + 2**96 - 1 #  prime defining the field
# y^ = x^3 + ax + b
a = -3
b = 41058363725152142129326129780047268409114441015993725554835256314039467401291
Gx = 0x6B17D1F2E12C4247F8BCE6E563A440F277037D812DEB33A0F4A13945D898C296
Gy = 0x4FE342E2FE1A7F9B8EE7EB4A7C0F9E162BCE33576B315ECECBB6406837BF51F5
n = 115792089210356248762697446949407573529996955224135760342422259061068512044369 # G's order

# Create the curve and the generator on sage
P256 = EllipticCurve(Zmod(p), [a, b])
G = P256([Gx, Gy])

# Public key to verify the signatures 
P = P256(0x7453873d3b072c061d03c3d09aba344b6a25f406a3dad40b0281025824bda4a3, 0x9e41b56ecaaf199fbb6d33fb2d748ec454c128aa7be74a1998cf57de34b7988b)


m2_1 = "Deux messages différents,  signés utilisant le même nonce "
r2_1 = 0xa6b2cbe4982361d0132b720c698ae5cb866271aa3f39b564018848a3bb7fbe07
s2_1 = 0x700f22bbdaa2027936bff58819fa674a2ff83ff120d18990b6cddc933b87fd89
m2_2 = "sont facilement reconnaissables par leur signature. "
r2_2 = 0xd76446168b18bc500812599efb6e37e9bf4467614f943aa55901ef1f3a1a57ef
s2_2 = 0x947016dd00edd5bab0a2c91dbeff335bbe4b22449c91cb20a3766ed80aa32bd1
m2_3 = "Il est alors possible de retrouver une information secrète..."
r2_3 = 0xa6b2cbe4982361d0132b720c698ae5cb866271aa3f39b564018848a3bb7fbe07
s2_3 = 0x1437dd0ebd91efcebe8bf7d93c8a4a5ba3b7a900bf1a1a37da20a3a1e7a4d8fa

# We can notice that r2_1 = r2_3, and r comes from the result of k*G. Hence, 
# r2_1 = r2_3 implies k2_1 = k2_3. From there, we can easily recover the key.
h1 = int(sha256(m2_1.encode()).digest().hex(), 16)
h3 = int(sha256(m2_3.encode()).digest().hex(), 16)
k = ((s2_1-s2_3).inverse_mod(n) * (h1-h3)) % n
d2 = (s2_1*k - h1)*r2_1.inverse_mod(n) % n
print("The private key is", hex(d2))

if d2*G == P:
    print("The private key we found fits the public key P")
else:
    print("The private key we found does not fit the public key P")


The private key is 0xe5f5a8f342537d97da4a1c6ff7014c1b4a9167c46e05779da1bcf172d49602a1
The private key we found fits the public key P
